In [59]:
import numpy as np
import networkx as nx
from routing_gym import RoutingEnv
import dgl
import torch
from networkx.algorithms.simple_paths import shortest_simple_paths

In [46]:
P = nx.petersen_graph()
print(P.number_of_edges())
dgl_P = dgl.DGLGraph(P)
print(dgl_P.number_of_edges())
env = RoutingEnv(P)

15
30


tensor([[1., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [55]:
def alloc_demand(graph, paths, bw):
    """
    graph: dgl graph
    path: list of node index (int)
    bw: bandwidth [8, 32, 64] (int)
    """
    for i, path in enumerate(paths):
        src = path[:-1] + path[1:]
        dst = path[1:] + path[:-1]
        graph.edata["feat"][graph.edge_ids(src, dst), i+1] = torch.zeros(len(src))+bw

    return graph


In [58]:
def step(act, paths, graph, nx_graph, k):
    """
    act: int corresponding with the chosen path to allocate the demand on
    """
    path = paths[act]
    src = path[:-1] + path[1:]
    dst = path[1:] + path[:-1]
    graph.edata["feat"][graph.edge_ids(src, dst), 0] -= bw
    src, dst = np.random.choice(nx_graph.number_of_nodes(), size=2, replace=False)
    shortest_paths = shortest_simple_paths(nx_graph, src, dst)[:k]
    bw = np.random.choice([8, 32, 64])
    alloc_demand(graph, shortest_paths, bw)

In [57]:
dgl_P = dgl.DGLGraph(P)
dgl_P.edata["feat"] = torch.zeros((dgl_P.number_of_edges(), 10))
alloc_demand(dgl_P, [[1,2,3], [1,2,7]], 1)
dgl_P.edata['feat']

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],


In [28]:
torch.zeros(4) + 1

tensor([1., 1., 1., 1.])